In [ ]:
from selenium import webdriver

from selenium.webdriver import Chrome
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from webdriver_manager.chrome import ChromeDriverManager
from bs4 import BeautifulSoup
from selenium.webdriver.chrome.options import Options
import pandas as pd
import math
import concurrent.futures
import os
import time

In [ ]:
options = webdriver.ChromeOptions()
options.add_argument('headless')
# changed from "none" to "normal" to enable full page loading...again thinking this the issue with repeating results
options.page_load_strategy='normal'
chrome_path = ChromeDriverManager().install()
chrome_service = Service(chrome_path)
driver = Chrome(options=options, service=chrome_service)
driver.implicitly_wait(5)

In [ ]:
#Created individual Race_Data files with each year to enable running in parallel sessions
racedetails = pd.read_csv(r'C:\Users\developer\Documents\NYC_Marathon_Data\NYRR_Race_Data_2019.csv', index_col=None)
print(racedetails)
# This loop steps through each year of the race
# It will wait until one race is completely finished before it loops through again
#   and creates a new set of threads for the next race. Will use this if we do multiple years at one time with ThreadPoolExecutor
for index, individual_race in racedetails.iterrows():
    event = individual_race['Event']
    year = individual_race['Year']
    dir = r"C:\Users\developer\Documents\NYC_Marathon_Data"
    finisher = individual_race['Finishers']
    pages = math.ceil(finisher / 51)
print(pages)

In [ ]:
# The range was originally (1, pages + 1, 1) i.e. start at 1 and go to pages plus 1 incrementing by 1.
# In the end it was easer to set of a bunch of different ranges in parallel to scrape a given event faster (in parallel)
for page in range (170, 200, 1):   
#To avoid stuff in memory giving the same results repeatedly, the driver was stopped (driver.quit()) and restarted in each round of the loop
    driver.quit()
    from selenium.webdriver import Chrome
    from selenium.webdriver.chrome.service import Service
    from selenium.webdriver.common.by import By
    from webdriver_manager.chrome import ChromeDriverManager
    from selenium.webdriver.chrome.options import Options
    options = webdriver.ChromeOptions()
    options.add_argument('headless')
    options.page_load_strategy='normal'
    chrome_path = ChromeDriverManager().install()
    chrome_service = Service(chrome_path)
    driver = Chrome(options=options, service=chrome_service)
    driver.implicitly_wait(5)
    place = ((page - 1) * 51) + 1
    fname = f"{dir}/{year}/{place}.csv"
    url = f'https://results.nyrr.org/event/{event}/finishers#opf={str(place)}&page=1'
    driver.get(url)
# content holds the entire list of divs; finisher represents an individual div
    content = driver.find_elements(By.CSS_SELECTOR, "div[ng-repeat*='eventFinisher in eventFinishers']")
    rows = pd.DataFrame(columns=['Name', 'Gender', 'Age', 'State', 'Country', 'Time', 'Overall'])

In [ ]:
    for finisher in content:
# Create a blank dictionary to hold the data
        newFinisher = {}
        name = finisher.find_elements(By.CSS_SELECTOR, "div[class*='name']")
        newFinisher["Name"] = name[0].text if name else ""
        genderage = finisher.find_elements(By.CSS_SELECTOR, "span[ng-if*='eventFinisher.gender']")
        if genderage:
            newFinisher["Gender"] = genderage[0].text[0] if genderage[0].text[0] in "MWX" else ""
            newFinisher["Age"] = genderage[0].text[1:] if genderage[0].text[0] in "MWX" else genderage[0].text
        else:
            newFinisher["Gender"] = ""
            newFinisher["Age"] = ""
        iaaf = finisher.find_elements(By.CSS_SELECTOR, "span[ng-if*='eventFinisher.iaaf']")
        if iaaf and iaaf[0].text == 'USA':
            state = finisher.find_elements(By.CSS_SELECTOR, "span[ng-if*='eventFinisher.stateProvince']")
            newFinisher["State"] = state[0].text if state else ""
            newFinisher["Country"] = iaaf[0].text if iaaf else ""
        else:
            state = None
            newFinisher["State"] = state[0].text if state else ""
            newFinisher["Country"] = iaaf[0].text if iaaf else ""
        results = finisher.find_elements(By.CSS_SELECTOR, "span[class*='result']")    
        for result in results:
            if ("Time" in result.text):
                newFinisher["Time"] = result.text.removeprefix("Time").strip()
            if("Place" in result.text):
                newFinisher["Overall"] = result.text.removeprefix("Place").replace(',', '').strip()
#having print indented at this location is faster
#             print(newfinisher)
# to check if the right page is subject to the driver.find_elements
#             print(url)
        rows = pd.concat([rows, pd.DataFrame.from_dict([newFinisher])], axis=0, ignore_index=True)
        rows.to_csv(fname, mode='w', index=False, header=True)

In [ ]:
#The files from a given event (year) were concatenated into one file with the lines below. The file was inspected for missing data 
# which was evident when the row number no longer aligned with the overall place (after sorting). Scraping for missing file was then 
#performed to catch the stragglers and the files were re-concatenated until the total number of finishers was as expected

import pandas as pd
import math
import os

# Each of the directory year, path to file and output name were changed for each year (make sure all the years align!) before running
dir = r"C:\Users\developer\Documents\NYC_Marathon_Data"
files = os.listdir(f"{dir}/2018")

rows = pd.concat([pd.read_csv(f"{dir}/2018/{file}") for file in files ], ignore_index=True) 
rows.to_csv(f"{dir}/2018results.csv", index=False, header=True)